In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search
)

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-19 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-19 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-19 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-19 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-19 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(9699, 384)

# swmhau Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [7]:
features = ["timeline_index"]
standardise_method = [None]
num_features = len(features)
add_time_in_path = True

In [8]:
num_epochs = 100
embedding_dim = 384
dimensions = [15] # [50, 15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6), (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

## UMAP

In [12]:
size = 20
swmhau_network_umap_kfold_20, best_swmhau_network_umap_kfold_20, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    embedding_dim=embedding_dim,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold.csv


In [13]:
swmhau_network_umap_kfold_20.groupby(["dimensions",
                                     "output_channels",
                                     "sig_depth",
                                     "num_heads",
                                     "num_layers",
                                     "ffn_hidden_dim",
                                     "learning_rate"]).mean()

/tmp/ipykernel_2372734/767646728.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_20.groupby(["dimensions",


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.610857   
                                                                         0.0005         0.672203   
                                                                         0.0010         0.668639   
                                                          (512, 512)     0.0001         0.615874   
                                                                         0.0005         0.668535   
                                                                         0.0010         0.673241   
                                     12        1          (256, 256)     0.0001         0.611722   
                                                                         0.0005         0.670057   
                                                                         0.0010         0.669954   
                                                          (512, 512)     0.0001         0.613279   
                                                                         0.0005         0.672306   
                                                                         0.0010         0.666563   
           12              3         10        1          (256, 256)     0.0001         0.630060   
                                                                         0.0005         0.659678   
                                                                         0.0010         0.671165   
                                                          (512, 512)     0.0001         0.627707   
                                                                         0.0005         0.652827   
                                                                         0.0010         0.671857   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.431180   
                                                                         0.0005         0.552062   
                                                                         0.0010         0.538557   
                                                          (512, 512)     0.0001         0.429892   
                                                                         0.0005         0.542491   
                                                                         0.0010         0.540363   
                                     12        1          (256, 256)     0.0001         0.423237   
                                                                         0.0005         0.544321   
                                                                         0.0010         0.543481   
                                                          (512, 512)     0.0001         0.429250   
                                                                         0.0005         0.549374   
                                                                         0.0010         0.534416   
           12              3         10        1          (256, 256)     0.0001         0.440991   
                                                                         0.0005         0.526698   
                                                                         0.0010         0.541992   
                                                          (512, 512)     0.0001         0.446202   
                                                                         0.0005         0.527309   
                                                                         0.0010         0.536973   

                                                                                      

In [14]:
best_swmhau_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.675524,0.553209,"[0.7921830314585321, 0.4988804299149127, 0.368...",0.555881,"[0.7805259862241704, 0.5279620853080569, 0.359...",0.551836,"[0.8041935483870968, 0.47283531409168084, 0.37...",None,0.669025,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.666805,0.552002,"[0.7854474581776839, 0.5037231712658782, 0.366...",0.548562,"[0.7909715407262021, 0.5203619909502263, 0.334...",0.558141,"[0.78, 0.48811544991511036, 0.40630797773654914]",None,0.658333,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.668258,0.546206,"[0.7858407079646018, 0.5017035775127768, 0.351...",0.546917,"[0.783948635634029, 0.5034188034188034, 0.3533...",0.545512,"[0.787741935483871, 0.5, 0.34879406307977734]",None,0.674371,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [15]:
best_swmhau_network_umap_kfold_20["f1"].mean()

0.550472221735998

In [16]:
best_swmhau_network_umap_kfold_20["precision"].mean()

0.550453175058193

In [17]:
best_swmhau_network_umap_kfold_20["recall"].mean()

0.5518296614318928

In [18]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.78782373, 0.50143573, 0.36215721])

In [19]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.78514872, 0.51724763, 0.34896318])

In [20]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.79064516, 0.48698359, 0.37786024])

## Random Projections

In [9]:
size = 20
swmhau_network_grp_kfold_20, best_swmhau_network_grp_kfold_20, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    embedding_dim=embedding_dim,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features,
    standardise_method=standardise_method,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_20_kfold_best_model.csv


In [10]:
swmhau_network_grp_kfold_20.groupby(["dimensions",
                                     "output_channels",
                                     "sig_depth",
                                     "num_heads",
                                     "num_layers",
                                     "ffn_hidden_dim",
                                     "dropout_rate",
                                     "learning_rate"]).mean()

/tmp/ipykernel_915499/1948036447.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_20.groupby(["dimensions",


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.604941   
                                                                                      0.0005         0.615874   
                                                                                      0.0010         0.664937   
                                                                         0.5          0.0001         0.610892   
                                                                                      0.0005         0.627846   
                                                                                      0.0010         0.664175   
                                                          (512, 512)     0.1          0.0001         0.579960   
                                                                                      0.0005         0.640855   
                                                                                      0.0010         0.664729   
                                                                         0.5          0.0001         0.602588   
                                                                                      0.0005         0.623694   
                                                                                      0.0010         0.666321   
                                     12        1          (256, 256)     0.1          0.0001         0.605425   
                                                                                      0.0005         0.611861   
                                                                                      0.0010         0.657394   
                                                                         0.5          0.0001         0.611169   
                                                                                      0.0005         0.635665   
                                                                                      0.0010         0.664383   
                                                          (512, 512)     0.1          0.0001         0.582382   
                                                                                      0.0005         0.623140   
                                                                                      0.0010         0.662653   
                                                                         0.5          0.0001         0.591585   
                                                                                      0.0005         0.598505   
                                                                                      0.0010         0.674555   
           12              3         10        1          (256, 256)     0.1          0.0001         0.615805   
                                                                                      0.0005         0.622517   
                                                                                      0.0010         0.625216   
                                                                         0.5          0.0001         0.626254   
                                                                                      0.0005         0.616013   
                                                                                      0.0010         0.616220   
                                                          (512, 512)     0.1          0.0001         0.617120   
                                                                                      0.0005         0.596914   
                                                                                      0.0010         0.616705   
                                                                         0.5          0.0001         0.616013   
                                 

In [11]:
best_swmhau_network_grp_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.660162,0.549994,"[0.7806072477962782, 0.48387096774193544, 0.38...",0.544710,"[0.7901520158625248, 0.49731182795698925, 0.34...",0.558855,"[0.7712903225806451, 0.47113752122241087, 0.43...",None,0.654717,...,0.001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.682582,0.554785,"[0.800626468285043, 0.48626373626373626, 0.377...",0.562681,"[0.7780821917808219, 0.5278330019880716, 0.382...",0.549398,"[0.824516129032258, 0.45076400679117146, 0.372...",None,0.674371,...,0.001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.652066,0.551685,"[0.7697780789429105, 0.5064883995281164, 0.378...",0.543014,"[0.8105601141633964, 0.4717948717948718, 0.346...",0.565677,"[0.7329032258064516, 0.5466893039049237, 0.417...",None,0.645126,...,0.001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [12]:
best_swmhau_network_grp_kfold_20["f1"].mean()

0.552154448583109

In [13]:
best_swmhau_network_grp_kfold_20["precision"].mean()

0.550135242097097

In [14]:
best_swmhau_network_grp_kfold_20["recall"].mean()

0.5579767005840254

In [15]:
np.stack(best_swmhau_network_grp_kfold_20["f1_scores"]).mean(axis=0)

array([0.7836706 , 0.4922077 , 0.38058505])

In [16]:
np.stack(best_swmhau_network_grp_kfold_20["precision_scores"]).mean(axis=0)

array([0.79293144, 0.4989799 , 0.35849439])

In [17]:
np.stack(best_swmhau_network_grp_kfold_20["recall_scores"]).mean(axis=0)

array([0.77623656, 0.48953028, 0.40816327])